In [0]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
# from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [0]:
# lectura de los datos
(val_X, val_Y), (train_X, train_Y) = mnist.load_data()

In [0]:
# pre-procesamiento de los datos

train_X = train_X.reshape(train_X.shape[0],28,28,1)
val_X = val_X.reshape(val_X.shape[0],28,28,1)
train_X = train_X.astype('float32')
val_X = val_X.astype('float32')

# normalizar datos de [0..255] a [0..1]
train_X /= 255
val_X /= 255

# convierte etiqueta de clase en "one-hot vectors" [0,0,1,0,...]
num_classes = 10
train_Y = keras.utils.to_categorical(train_Y, num_classes)
val_Y = keras.utils.to_categorical(val_Y, num_classes)

**Efectos visibles de técnicas de regularización**
* Sólo Dropout: p=0.5 tras Flatten + p=0.25 tras Dense(100)
* Sólo MaxPooling: (2,2) tras cada capa Conv2D
* Dropot + MaxPolling: (2,2) tras cada capa Conv2D + p=0.25 tras Flatten y Dense(100)

In [0]:
# modelo: capa entrada + 20 conv 5x5 + 40 conv 5x5 + densa(100) + capa densa salida (10)

model = Sequential()

model.add(Conv2D(20, kernel_size=5, activation='relu', input_shape=(28,28,1))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(40, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

# imprime resumen del modelo
model.summary()

In [0]:
# hiperparámetros del proceso de entrenamiento
batch_size = 100
epochs = 20

# optimizador (actualización de pesos)
opt=SGD(lr=0.05, decay=1e-6, momentum=0.9)

# configuración del proceso de entrenamiento
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# entrenamiento del modelo
history = model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(val_X, val_Y))

In [0]:
# resultados sobre entrenamiento
train_results = model.evaluate(train_X, train_Y, verbose=0)
print('Error (loss) sobre datos de entrenamiento: {:.4f}'.format(train_results[0]))
print('Tasa de éxito de clasificación (accuracy) sobre datos de entrenamiento: {:.4f}\n'.format(train_results[1]))

# resultados sobre validación
val_results = model.evaluate(val_X, val_Y, verbose=0)
print('Error (loss) sobre datos de VALIDACIÓN: {0:.4f}'.format(val_results[0]))
print('Tasa de éxito de clasificación (accuracy) sobre datos de VALIDACIÓN: {0:.4f}\n'.format(val_results[1]))